In [1]:
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn import neighbors

import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
train = pickle.load(open('train_processed.pkl','rb'))
train.head()

,Dates,Category,Address,X,Y,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,2015-05-13 23:53:00,WARRANTS,OAK ST / LAGUNA ST,-122.425892,37.774599,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2015-05-13 23:53:00,OTHER OFFENSES,OAK ST / LAGUNA ST,-122.425892,37.774599,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-13 23:33:00,OTHER OFFENSES,VANNESS AV / GREENWICH ST,-122.424363,37.800414,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2015-05-13 23:30:00,LARCENY/THEFT,1500 Block of LOMBARD ST,-122.426995,37.800873,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2015-05-13 23:30:00,LARCENY/THEFT,100 Block of BRODERICK ST,-122.438738,37.771541,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
test = pickle.load(open('test_processed.pkl','rb'))
test.head()

,Dates,Address,X,Y,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
Id,,,,,,,,,,,,,,
0,2015-05-10 23:59:00,2000 Block of THOMAS AV,-122.399588,37.735051,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-05-10 23:51:00,3RD ST / REVERE AV,-122.391523,37.732432,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-10 23:50:00,2000 Block of GOUGH ST,-122.426002,37.792212,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2015-05-10 23:45:00,4700 Block of MISSION ST,-122.437394,37.721412,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-05-10 23:45:00,4700 Block of MISSION ST,-122.437394,37.721412,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

In [3]:
# encode the category
trainset = pd.read_csv('train.csv', parse_dates = ['Dates'])
testset = pd.read_csv('test.csv', parse_dates = ['Dates'])
label = preprocessing.LabelEncoder()
crime = label.fit_transform(trainset.Category) 
train = pd.concat([pd.get_dummies(trainset.DayOfWeek), pd.get_dummies(trainset.PdDistrict), pd.get_dummies(trainset.Dates.dt.hour)], axis=1) 
train['crime'] = crime   
test = pd.concat([pd.get_dummies(testset.DayOfWeek), pd.get_dummies(testset.PdDistrict), pd.get_dummies(testset.Dates.dt.hour)], axis=1)  

In [ ]:
trainset.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [ ]:
train.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,crime
0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37
1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21
2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21
3,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16
4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16


In [6]:
# split into training set and test set
training, validation = train_test_split(train, train_size=0.3)# TODO: gridsearchCV

feature_list = training.columns.tolist()   
feature_list = feature_list[:len(feature_list) - 1] 

## Bayesian Model

In [ ]:
# Naïve Bayes Model
nb_model = GaussianNB()
nb_model.fit(training[feature_list], training['crime'])
print('Naïve Bayes accuracy for train set: %f' % nb_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(nb_model.predict_proba(test[feature_list])) 

col = np.sort(trainset['Category'].unique())  
result = pd.DataFrame(data = test_predicted, columns = col)  
result['Id'] = testset['Id'].astype(int) 
result.to_csv('GaussianNB.csv', index=False)  

Naïve Bayes accuracy for train set: 0.001979


In [7]:
nb_model = BernoulliNB()
nb_model.fit(training[feature_list], training['crime'])   
predicted = np.array(nb_model.predict_proba(validation[feature_list]))   
print('Naïve Bayes accuracy for train set: %f' % nb_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(nb_model.predict_proba(test[feature_list])) 

col = np.sort(trainset['Category'].unique())  
result = pd.DataFrame(data = test_predicted, columns = col)  
result['Id'] = testset['Id'].astype(int) 
result.to_csv('BernoulliNB.csv', index=False)  

Naïve Bayes accuracy for train set: 0.222949


## Logistic Regression Model

In [ ]:
# TODO: test different C values
lr_model= LogisticRegression(C=0.5)
lr_model.fit(training[feature_list], training['crime'])
predicted = np.array(lr_model.predict_proba(validation[feature_list]))
print('Logistic Regression accuracy for train set: %f' % lr_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(lr_model.predict_proba(test[feature_list])) 

col = np.sort(trainset['Category'].unique())  
result = pd.DataFrame(data = test_predicted, columns = col)  
result['Id'] = testset['Id'].astype(int) 
result.to_csv('LR.csv', index=False)  

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression accuracy for train set: 0.223131


## Random Forest Model

In [ ]:
rf_model = RandomForestClassifier()  
rf_model.fit(training[feature_list], training['crime'])
predicted = np.array(rf_model.predict_proba(validation[feature_list]))
print('Random Forest accuracy for train set: %f' % rf_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(rf_model.predict_proba(test[feature_list])) 

col = np.sort(trainset['Category'].unique())  
result = pd.DataFrame(data = test_predicted, columns = col)  
result['Id'] = testset['Id'].astype(int) 
result.to_csv('RF.csv', index=False) 

Random Forest accuracy for train set: 0.229716


## KNN model

In [9]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 20)
knn_model = knn.fit(training[feature_list], training['crime'])
print('NN accuracy for train set: %f' % knn_model.score(training[feature_list], training['crime']))

NN accuracy for train set: 0.199016


In [10]:
# test on the test set 
test_predicted = np.array(knn_model.predict_proba(test[feature_list])) 

col = np.sort(trainset['Category'].unique())  
result = pd.DataFrame(data = test_predicted, columns = col)  
result['Id'] = testset['Id'].astype(int) 
result.to_csv('KNN_20.csv', index=False)  

In [ ]:
print('NN accuracy for train set: %f' % knn_model.score(training[feature_list], training['crime']))

NN accuracy for train set: 0.171740
